# Projet CENSE

In [3]:
import librosa.display
import IPython.display as ipd

# feature extractoring and preprocessing data
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os

from PIL import Image
import pathlib

# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Normalization
from sklearn import preprocessing 

import warnings
warnings.filterwarnings('ignore')

# prprocessing

In [4]:
np.random.seed(43)
file_path = '.\TUT-acoustic-scenes-2017-development.meta\TUT-acoustic-scenes-2017-development\evaluation_setup'
fold1 = os.listdir(file_path)

In [5]:
train = os.path.join(file_path,fold1[2])
fold1_filename_train = []
fold1_label_train = []
with open(train, 'r') as f:
    data = f.readlines()  #data reading line by line
    for line in data:
        record = line.split()        #split the filename and label
        fold1_filename_train.append(record[0][6:])
        fold1_label_train.append(record[1])
print('fold 1 has {} sounds to train'.format(len(fold1_filename_train)))

fold 1 has 3510 sounds to train


Because training set is too large, so we can take out some songs for testing our method

In [6]:
audio_train = []

indices = np.arange(len(fold1_filename_train))
np.random.shuffle(indices)
n_sound = len(fold1_filename_train)  # select some data to have a try, otherwise we consider the whole data set
indices = indices[:n_sound]
fold1_filename_train = np.array(fold1_filename_train)[indices]
fold1_label_train = np.array(fold1_label_train)[indices]

for root,dirnames,filenames in os.walk('./'):
    for filename in filenames:
         if filename in fold1_filename_train:
            f = os.path.join(root, filename)
            audio_train.append(f)
print(f'training set has {len(audio_train)} data')
print(f'training set label has {len(fold1_label_train)} records')
classes_train = set(fold1_label_train)  # find the unique elements
n_classes_train = len(classes_train)
print(f'There are {n_classes_train} classes in the training set')

training set has 3510 data
training set label has 3510 records
There are 15 classes in the training set


In [7]:
dictionary = dict(zip(classes_train, list(np.arange(n_classes_train))))  # build the connection between labels and numbers
print('label list :\n')
for item in classes_train:
    print(item, dictionary[item])

label list :

tram 0
residential_area 1
home 2
library 3
car 4
beach 5
forest_path 6
train 7
office 8
grocery_store 9
cafe/restaurant 10
park 11
bus 12
city_center 13
metro_station 14


In [8]:
temp=[]
for label in fold1_label_train:
    temp.append(dictionary[label])
fold1_label_train= temp

It takes a lot of time to calculate the MFCC coefficients of traning set, ***30 min*** to load 3510 sound files

In [9]:
%%time

# MFCC is a matrix who stores the MFCC coefficients of each audio

if os.path.exists('MFCC.npy'):
    MFCC = np.load('MFCC.npy')
else:
    MFCC = np.zeros((n_sound,20,431))
    i = 0
    for song,label in zip(audio,fold1_label_train):
        y, sr = librosa.load(song, mono=True)    # original audio file lasts for 10 seconds
        MFCC[i,:,:]= librosa.feature.mfcc(y=y, sr=sr)    # 20 * 431
        i+=1
    np.save('MFCC.npy',MFCC)
print('MFCC shape is ',MFCC.shape)

MFCC shape is  (3510, 20, 431)
Wall time: 3.03 s


# fenêtre de texture

In [32]:
# x and y should be vectors
def dist(x,y,choix):
    x_ = (x-np.mean(x))/np.std(x)
    y_ = (y-np.mean(y))/np.std(y)
    if choix == "euclidian":
        return(np.linalg.norm(x_-y_))
    elif choix=="cosine":
        return(1- np.vdot(x_,y_)/np.linalg.norm(x_)/np.linalg.norm(y_))
    
# x and y are feature matrix, the function will calculate the temporal mean MFCC coefficients
def dist_moy(x,y,choix):
    """x et y sont des matrices de MFCC. On calcule la distance entre x et y en faisant la 
    moyenne de chaque dans les matrices puis on regarde la distance entre les deux vecteurs de moyennes."""
    
    xbarre = np.mean(x,axis=1)
    ybarre = np.mean(y,axis=1)
    output = dist(xbarre,ybarre,choix)
    return(output)

def wind_texture(M,wsize):
    """Calcul les moyennes temporelles des fenêtres de textures dans une matrice de MFCC.

    M : Matrice de MFCC au format Quefrence x temps
    wsize : Nombre de trames retenues dans la fenêtre"""

    # n est le nombre de caractéristiques, m est la nombre de trame
    n,m = M.shape
    nwind = m//wsize
    
    W = np.zeros((n,nwind))
    
    for i in range (nwind):
        W[:,i] = np.mean(M[:,i*wsize:(i+1)*wsize],axis=1)
    
    return(W)

def dist_wind(A,B,choix,wsize):
    """La distance entre A et B correspond à la distance entre les deux fenêtres les plus proches.
    
    A,B : Matrices de MFCC
    choix : distance euclidienne ou cosinus"""
    
    x = wind_texture(A,wsize)
    y = wind_texture(B,wsize)
    
    n,m = np.shape(x)
    distxy = np.zeros((m,1))
    
    for k in range(m):
        distxy[k] = dist(x[:,k],y[:,k],choix)
    return(np.min(distxy))
#     return(distxy[np.where(distxy==min(distxy))[0][0]])

def p_at_k_index(m,mat_dist,k):

# for m-th scene, return the index which indicate the k nearest neighbors

    N_train = mat_dist.shape[0]
    p_k = np.zeros(N_train)
    dist = mat_dist[m,:]
    distsort = np.argsort(np.ravel(dist)) # from small to big
    return distsort[1:k+1]

def centroid(S):
    # S is the MFCC column
#     freq = np.linspace(0,sr/2,nfft/2+1)
    freq = np.linspace(10,1024,20)
    freq = np.log(freq)
    centroid = np.sum(S*freq)/np.sum(S) 
    return centroid

In [11]:
# Normalization on each segment of MFCC matrix
N_train = len(audio_train)
data_train = MFCC[:,1:,:]   # eliminate the first MFCC coefficient which indicates the energy of each segment
data_train_norm = np.zeros(np.shape(data_train))
for i in range(N_train):
    data_ = data_train[i,:,:]
    data_train_norm[i,:,:] = (data_- np.mean(data_,axis=0))/np.std(data_,axis=0)
print('training data format: ',data_train_norm.shape)

training data format:  (3510, 19, 431)


In [ ]:
# test about spectral centroid
nfft = 2048
song = audio_train[0]
y, sr = librosa.load(song, mono=True)
S, phase = librosa.magphase(librosa.stft(y=y))
centroid = librosa.feature.spectral_centroid(y=y,sr=sr)
freq = np.linspace(0,sr/2,nfft/2+1).reshape(-1,1)
freq = np.tile(freq,(1,np.shape(S)[1]))
cen = np.sum(S*freq,axis=0)/np.sum(S,axis=0)  # equal to centroid

In [18]:
%%time
if os.path.exists('mat_dist_moy.npy'):
    mat_dist_moy = np.load('mat_dist_moy.npy')
else:
    mat_dist_moy = np.zeros((N_train,N_train))  # calculate the distance among all 3510 files
    for i in range(N_train):
        for j in range(i,N_train):
            mat_dist_moy[i,j] = dist_moy(data_train_norm[i],data_train_norm[j],'euclidian')
            mat_dist_moy[j,i] = mat_dist_moy[i,j]
    # 20 min to calculate the disrtance
    np.save('mat_dist_moy.npy', mat_dist_moy)

Wall time: 1.27 s


In [39]:
def RBF(M, u, v, mat_dist, wsize,K=5, type = 'RBQ-c'):
    """
    Input:
    M: MFCC matrix which represent all scenes in the training set
    u,v: u-th and v-th scene index,respectively which we will analyze
    mat_dist: distance between every two scenes in the M matrix 
    labels: scene type in the M matrix
    K: number of nearest neighbors
    type: kernel type
    
    Output:
    similarity:  imilarity between the scene centroids over the entire dataset
    
    Attention:
    1.Here, μ_um_q and μ_vn_q are the q-th nearest neighbors to the centroids μ_um and μ_vn, respectively
    2.By default the Euclidean norm is to be used.
    3.Three types to be selected: RBQ-c, RBQ-a, RBQ-w, by default RBQ-c
    4.By default we consider K-nearest neignbors K=5
    """
    U = M[u,:,:]   # feature matrix of each scene
    V = M[v,:,:]
    
    μ_um = wind_texture(U,wsize)  # window filtered feature matrix 
    μ_vn = wind_texture(V,wsize)
    
    d,m = μ_um.shape   # after window, d: feature number; m,n: segment number, m=n
    d,n = μ_vn.shape
    
    u_knn = p_at_k_index(u,mat_dist,K)  # index
    v_knn = p_at_k_index(v,mat_dist,K)
    
    K_mn = np.zeros((K,m,n)) # K nearest neighbors->K elements
    
    for q in range(K):
        μ_um_q = M[u_knn[q],:,:]  # feature matrix of q-th nearest neighbors for each scene
        μ_vn_q = M[v_knn[q],:,:]
        
        μ_um_q = wind_texture(μ_um_q,wsize)  # window filtered feature matrix 
        μ_vn_q = wind_texture(μ_vn_q,wsize)
        
        for mm in range(m):
            for nn in range(n):
                UM = μ_um[:,mm]
                VN = μ_vn[:,nn]
                UMQ = μ_um_q[:,mm]
                VNQ = μ_vn_q[:,nn]
                num = centroid(UM - VN)
                den = (centroid(UM - UMQ))*(centroid(VN - VNQ))
                R = num*2 / den
                K_mn[q,mm,nn] = np.exp(-R)
        
    if type=='RBQ-c':
        temp = np.max(K_mn,axis = 1)
        similarity = np.max(temp,axis = 1)
    elif type == 'RBQ-a':
        temp = np.mean(K_mn,axis = 1)
        similarity = np.mean(temp,axis = 1)
    elif type == 'RBQ-w':
        # Suppose in the matrix μ_um there are M columns so there will be M centroids
        # each of the M centroids μu1, . . . , μuM are paired with corresponding weights wu1, . . . , wuM
        # The weight wum for the m-th centroid μum is the number of frames belonging to a particular cluster
        # Since wh use the windows with equal length so the weight is the same for each columm
        
        w = np.ones((K,m,n))/(m*n)
        temp = np.sum(K_mn*w,axis=1)
        similarity = np.sum(temp,axis=1)
    else:
        return(print('Wrong type!'))
    return(similarity)

In [40]:
RBF(MFCC, 1, 2, mat_dist_moy, 21,K=5, type = 'RBQ-c')

array([2.46579221e+123, 1.94563408e+129, 4.06307248e+075, 3.53857383e+103,
       5.24892161e+006])

In [44]:
# matrix rbf shows the similarity between every two scene in the training set

# nb_train = len(audio_train)
# rbf = np.zeros((nb_train,nb_train))
# for u in range(nb_train):
#     for v in range(nb_train):
#         rbf[u,v] = RBF(MFCC,u,v,mat_dist_moy, 21,K=5, type = 'RBQ-c').max()


In [305]:
test = os.path.join(file_path,fold1[0])
fold1_filename_test = []
fold1_label_test = []

with open(test, 'r') as f:
    data = f.readlines()  #data reading line by line
 
    for line in data:
        record = line.split()        #split the filename and label
        fold1_filename_test.append(record[0][6:])
        fold1_label_test.append(record[1])
print('fold 1 has {} documents to test'.format(len(fold1_filename_test)))


fold 1 has 1170 documents to test


In [306]:
audio_test = []

for root,dirnames,filenames in os.walk('./'):
    for filename in filenames:
         if filename in fold1_filename_test:
            f = os.path.join(root, filename)
            audio_test.append(f)

print(f'Test set has {len(audio_test)} records')
classes_test = set(fold1_label_test)
n_classes_test = len(classes_test)
print(f'There are {n_classes_test} classes in the test set')

Test set has 1170 records
There are 15 classes in the test set


In [307]:
temp=[]
for label in fold1_label_test:
    temp.append(dictionary[label])
fold1_label_test= temp

It takes a lot of time to calculate the MFCC coefficients of test set, about 11m 7s to finish the calculation.

In [308]:
%%time
# M_test is a matrix who stores the MFCC infomation about each test audio

if os.path.exists('M_test.npy'):
    M_test = np.load('M_test.npy')
else:
    M_test = np.zeros((len(audio_test),20,431))
    i = 0
    for song in audio_test:
        y, sr = librosa.load(song, mono=True) 
        M_test[i,:,:] = librosa.feature.mfcc(y=y, sr=sr)
        i+=1 
    np.save('M_test.npy',M_test)   

Wall time: 1.26 s


In [309]:
# Normalization on each segment of MFCC matrix

N_test = len(audio_test)
data_test = M_test[:,1:,:]  # eliminate the first MFCC coefficient which indicates the energy of each segment
data_test_norm = np.zeros(np.shape(data_test))

for i in range(N_test):
    data_ = data_test[i,:,:]
    data_test_norm[i,:,:] = (data_-np.mean(data_,axis=0))/np.std(data_,axis=0)
print('test data format: ',data_test_norm.shape)

test data format:  (1170, 19, 431)


In [310]:
print(data_test_norm.shape)
print(data_train_norm.shape)
print(len(fold1_label_train))
print(len(fold1_label_test))

(1170, 19, 431)
(3510, 19, 431)
3510
1170
